In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

In [2]:
# Load XRF data
xrf = pd.read_csv("xrf/20892_xrf.csv")
xrf.columns = xrf.columns.str.strip()
xrf["Depth"] = xrf["Depth"].str.strip()
xrf["Component"] = xrf["Component"].str.strip()
xrf["depth_cm"] = xrf["Depth"].str.extract(r"(\d+)").astype(int)
xrf["source"] = "XRF"

# Pivot to wide then back to long to keep it simple
xrf_long = xrf[["depth_cm", "Component", "Result", "Unit", "source"]].copy()
xrf_long.columns = ["depth_cm", "element", "value", "unit", "source"]

# Load CHNS data
chns = pd.read_csv("chns/20892_chns.csv")
chns["depth_cm"] = chns["sample name"].str.extract(r"(\d+)cm").astype(int)

# Melt CHNS columns into long format
chns_elements = [c for c in chns.columns if c.endswith("%")]
chns_long = chns.melt(
    id_vars=["depth_cm"],
    value_vars=chns_elements,
    var_name="element",
    value_name="value",
)
chns_long["element"] = chns_long["element"].str.replace("%", "")
chns_long["unit"] = "wt%"
chns_long["source"] = "CHNS"

# Melt CHNS ratios (C/N, C/H)
chns_ratios = chns.melt(
    id_vars=["depth_cm"],
    value_vars=["C/N ratio", "C/H ratio"],
    var_name="element",
    value_name="value",
)
chns_ratios["element"] = chns_ratios["element"].str.replace(" ratio", "")
chns_ratios["unit"] = "molar"
chns_ratios["source"] = "CHNS"

# Compute C/S and C/Fe ratios per replicate using CHNS C% and XRF S%, Fe% at matching depths
# First get per-depth C from CHNS replicates and S, Fe from XRF
xrf_pivot = xrf_long.pivot_table(index="depth_cm", columns="element", values="value").reset_index()
chns_mean = chns.groupby("depth_cm")[["C%"]].mean().reset_index()
chns_mean.rename(columns={"C%": "C"}, inplace=True)

extra_ratios = []
for _, row in chns_mean.iterrows():
    d = row["depth_cm"]
    c_val = row["C"]
    xrf_row = xrf_pivot[xrf_pivot["depth_cm"] == d]
    if xrf_row.empty:
        continue
    if "S" in xrf_row.columns:
        s_val = xrf_row["S"].values[0]
        if s_val > 0:
            extra_ratios.append({"depth_cm": d, "element": "C/S", "value": c_val / s_val, "unit": "wt ratio", "source": "CHNS/XRF"})
    if "Fe" in xrf_row.columns:
        fe_val = xrf_row["Fe"].values[0]
        if fe_val > 0:
            extra_ratios.append({"depth_cm": d, "element": "C/Fe", "value": c_val / fe_val, "unit": "wt ratio", "source": "CHNS/XRF"})

extra_ratios_df = pd.DataFrame(extra_ratios)

# Combine all
df = pd.concat([xrf_long, chns_long, chns_ratios, extra_ratios_df], ignore_index=True)
all_elements = sorted(df["element"].unique())
print(f"Available elements/ratios: {all_elements}")

Available elements/ratios: ['Ag', 'Al', 'Br', 'C', 'C/Fe', 'C/H', 'C/N', 'C/S', 'Ca', 'Cl', 'Cu', 'Fe', 'H', 'K', 'Mg', 'Mn', 'N', 'Na', 'P', 'Rb', 'Rh', 'S', 'Si', 'Sr', 'Ti', 'Zn', 'Zr']


In [ ]:
def plot_element(element):
    subset = df[df["element"] == element].copy()
    if subset.empty:
        print(f"No data for {element}")
        return

    # Average replicates per depth
    stats = subset.groupby("depth_cm")["value"].agg(["mean", "std"]).reset_index()
    stats = stats.sort_values("depth_cm")
    unit = subset["unit"].iloc[0]
    source = subset["source"].iloc[0]

    fig, ax = plt.subplots(figsize=(4, 5))
    ax.errorbar(
        stats["mean"],
        stats["depth_cm"],
        xerr=stats["std"],
        fmt="o-",
        capsize=3,
        color="#1f77b4",
    )
    xmax = (stats["mean"] + stats["std"].fillna(0)).max()
    ax.set_xlim(0, xmax * 1.1)
    ax.invert_yaxis()
    ax.set_ylabel("Depth (cm)")
    ax.set_xlabel(f"{element} ({unit})")
    ax.set_title(f"{element} vs Depth (core 20892, {source})")
    plt.tight_layout()
    plt.show()


dropdown = widgets.Dropdown(
    options=all_elements,
    value="Fe",
    description="Element:",
)
widgets.interact(plot_element, element=dropdown);

interactive(children=(Dropdown(description='Element:', index=11, options=('Ag', 'Al', 'Br', 'C', 'C/Fe', 'C/H'…